<a href="https://colab.research.google.com/github/gitgitho0ray/scrappy/blob/master/10_times_scrappy_doo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from time import strptime
from time import sleep
import datetime
from datetime import datetime
from calendar import monthrange
from datetime import timedelta

In [0]:
class EventDate():

    def __init__(self, eventdate):

        if len(eventdate) == 4:
            sday=int(strptime(eventdate[0],'%d').tm_mday)
            month=int(strptime(eventdate[2],'%b').tm_mon)
            eday=int(strptime(eventdate[1],'%d').tm_mday)
            year=int(strptime(eventdate[-1],'%Y').tm_year)
            self.startdate = datetime.date(datetime(year, month, sday))
            self.enddate = datetime.date(datetime(year, month, eday))

        elif len(eventdate) == 5:
            sday=int(strptime(eventdate[0],'%d').tm_mday)
            smonth=int(strptime(eventdate[1],'%b').tm_mon)
            eday=int(strptime(eventdate[2],'%d').tm_mday)
            emonth=int(strptime(eventdate[3],'%b').tm_mon)
            year=int(strptime(eventdate[-1],'%Y').tm_year)
            self.startdate = datetime.date(datetime(year, smonth, sday))
            self.enddate = datetime.date(datetime(year, emonth, eday))

        else:
            sday=int(strptime(eventdate[0],'%d').tm_mday)
            month=int(strptime(eventdate[1],'%b').tm_mon)
            eday=int(strptime(eventdate[0],'%d').tm_mday)
            year=int(strptime(eventdate[-1],'%Y').tm_year)
            self.startdate = datetime.date(datetime(year, month, sday))
            self.enddate = datetime.date(datetime(year, month, eday))
        
class SearchDate():   

    def __init__(self,today='',endofweek='',endofthemonth=''):
        self.today=str(datetime.now().date())
        self.addoneweek=str((datetime.now().date()+timedelta(days=7)))
        self.endofthemonth=str(datetime(datetime.now().year,datetime.now().month,monthrange(datetime.now().year, datetime.now().month)[1]).date())

class ScrappyDoo():

    def __init__(self,url):
        self.url = url

    def get_content(self):

        def get_html(url, params=None):
            h = {'user-agent':'Mozilla/5.0 (Macintosh; Intel) Gecko/20100101 Firefox/74.0'}
            response = requests.get(self.url,params=params,headers=None)
            return response.text

        soup = BeautifulSoup(get_html(url), 'html.parser')
        items = soup.find_all('tr', class_='box')
        events=[i.find('a', {'target':'_blank'}).get('href') for i in items if i.find('a', {'target':'_blank'}) !=None]
        return events
        

    def parse():
        html = get_html(url)
        if html.status_code == 200:
            get_content(html.text)
        else:
            print('error')
        return html

In [0]:
if __name__ == '__main__':

    def get_html(url, params=None):
        h = {'user-agent':'Mozilla/5.0 (Macintosh; Intel) Gecko/20100101 Firefox/74.0'}
        response = requests.get(url,params=params,headers=None)
        return response.text
        
    date = SearchDate()
    url = f'https://10times.com/canada/conferences?month=today&datefrom={date.today}&dateto={date.today}'
    links = ScrappyDoo(url = url)

In [0]:
datalist=[]
for url in links.get_content(): 
    print(url)
    soup = BeautifulSoup(get_html(url), 'html.parser')

    #event info 
    eventname = soup.find('h1').get_text()

    try:
        organizer = soup.find('h3',{'id':'org-name'}).get_text().split('\n')[0]
    except AttributeError:
        organizer = soup.find('h3').get_text().split('\n')[0]

    description = soup.find('p', class_="desc mng word-break").get_text(strip=True)
    location=[i.get_text() for i in soup.find_all('p') if i.find('span') != None][0]
    eventdate = EventDate(soup.select('span[content]')[0].get_text().replace('-','').split())
    rawtables = [i.find('table',class_='table noBorder mng').find_all('td') for i in soup.find_all('div', class_='row11')]
    info=[rawtables[0][i].get_text().split(" ",1) for i in range(len(rawtables[0]))]
    infodata=[i[1].strip('\n').strip() for i in info[:4]]
    time = [" ".join(i[:2]) for i in [i.split() for i in infodata[0].split('-')]] # time 
    participants =(re.findall(r"[0-9]+\s-\s[0-9]+ | [0-9]+", infodata[2]))[0].strip()
    tags = (', '.join(infodata[3].replace('Type','').replace('&','').split()))
    
    #attendees data 
    users = soup.find('div', class_="visitor clearfix")
    attendeedata=[]
    if users is None:
        attendeedata.append(None)
    else:
        try:
            for i in soup.find('div', class_="visitor clearfix"):
                try:
                    attendeedata.append([i.find('h4').get_text(),' '.join(i.get_text().replace('Connect','').split()[-2:]),i.find('a').get('href')])
                except AttributeError:
                    continue
        except TypeError:
            continue
    
    datalist.append([url,
                   eventname,
                   organizer,
                   description,
                   location,
                   eventdate.startdate,
                   eventdate.enddate,
                   time[0],
                   time[1],
                   tags,
                   participants,
                   attendeedata])
    print('Processed')

df = pd.DataFrame(datalist,columns=['Link',
                                    'EventName',
                                    'Organizer',
                                    'Description',
                                    'Address',
                                    'StartDate',
                                    'EndDate',
                                    'StartTime',
                                    'EndTime',
                                    'Tags',
                                    'ExpectedParticipants',
                                    'AttendeeData'])
df